<a href="https://colab.research.google.com/github/chihchao/application-of-programming/blob/main/web_api_linebot_ask_gemini.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Final Ver

In [1]:
!pip install -q flask pyngrok flask_ngrok line-bot-sdk
# !pip install -q google-generativeai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 776.1/776.1 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.6/137.6 kB 4.7 MB/s eta 0:00:00


In [5]:
# for Colab
from pyngrok import ngrok
from flask_ngrok import run_with_ngrok
from google.colab import userdata
def ngrok_start():
    ngrok.set_auth_token(userdata.get('NGROK_AUTHTOKEN'))
    public_url = ngrok.connect(5000)
    run_with_ngrok(app)

# for linebot
from flask import Flask, request, abort

# LINE v3 SDK
from linebot.v3 import WebhookHandler
from linebot.v3.exceptions import InvalidSignatureError
from linebot.v3.webhooks import MessageEvent, TextMessageContent
from linebot.v3.messaging import (
    Configuration, ApiClient,
    MessagingApi, ReplyMessageRequest, TextMessage
)

# Google Gemini
from google import genai


app = Flask(__name__)

configuration = Configuration(access_token=userdata.get('LINE_CHANNEL_ACCESS_TOKEN'))
handler = WebhookHandler(userdata.get('LINE_CHANNEL_SECRET'))



# === Webhook 端點 ===
@app.route("/callback", methods=['POST'])
def callback():
    signature = request.headers['X-Line-Signature']
    body = request.get_data(as_text=True)
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        app.logger.info("Invalid signature. Please check your channel access token/channel secret.")
        abort(400)
    return 'OK'

# === 訊息事件處理 ===
@handler.add(MessageEvent, message=TextMessageContent)
def handle_message(event):
    with ApiClient(configuration) as api_client:
        reply_message = ask_gemini(event.message.text)
        line_bot_api = MessagingApi(api_client)
        line_bot_api.reply_message(
            ReplyMessageRequest(
                reply_token=event.reply_token,
                messages=[TextMessage(text=reply_message)]
            )
        )

GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
def ask_gemini(prompt: str) -> str:
    client = genai.Client(api_key=GOOGLE_API_KEY)
    response = client.models.generate_content(
        model="gemini-2.0-flash", contents=prompt
    )
    return response.text


ngrok_start()
if __name__ == "__main__":
    app.run()



 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


 * Running on http://b71e-34-75-169-4.ngrok-free.app
 * Traffic stats available on http://127.0.0.1:4040


INFO:werkzeug:127.0.0.1 - - [23/Apr/2025 04:12:18] "GET / HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [23/Apr/2025 04:12:19] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [23/Apr/2025 04:12:39] "POST /callback HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [23/Apr/2025 04:13:14] "POST /callback HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [23/Apr/2025 04:19:12] "POST /callback HTTP/1.1" 200 -


```javascript
{
  "destination": "Uc9d....",         // 回應目的地
  "events": [
    {
      "type": "message",             // 事件類型 join/leave, postback
      "message": {
        "type": "text",              // 訊息類型 sticker, image
        "id": "5579....",            // 這則訊息的 id
        "quoteToken": "LIke....",
        "text": "hello"              // 訊息內容
      },
      "webhookEventId": "01JS....",
      "deliveryContext": {
        "isRedelivery": false
      },
      "timestamp": 1745377105825,    // 發送訊息時間
      "source": {
        "type": "user",              // 訊息來源是 user 或 group
        "userId": "U1a8...."         // 傳送訊息的 userid
      },
      "replyToken": "ac52...",       // 每則訊息回應使用的 Token
      "mode": "active"
    }
  ]
}
```

## First Ver

In [ ]:
!pip install flask
!pip install pyngrok
!pip install flask_ngrok
!pip install line-bot-sdk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 779.9/779.9 kB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.6/137.6 kB 9.4 MB/s eta 0:00:00


In [ ]:
from flask import Flask, request, abort
# for  colab
from pyngrok import ngrok
from flask_ngrok import run_with_ngrok
from google.colab import userdata

def ngrok_start():
    ngrok.set_auth_token(userdata.get('NGROK_AUTHTOKEN'))
    public_url = ngrok.connect(5000)
    run_with_ngrok(app)

app = Flask(__name__)
ngrok_start()


import google.generativeai as genai
def ask_gemini(prompt):
    genai.configure(api_key=userdata.get('GOOGLE_API_KEY'))
    model = genai.GenerativeModel("gemini-2.0-flash")
    response = model.generate_content(prompt)
    return response.text

# for line bot
from linebot import (
    LineBotApi, WebhookHandler
)
from linebot.exceptions import (
    InvalidSignatureError
)
from linebot.models import (
    MessageEvent, TextMessage, StickerMessage, TextSendMessage, StickerSendMessage, ImageSendMessage, LocationSendMessage
)

def line_bot_start():
    line_bot_api = LineBotApi(userdata.get('LINE_CHANNEL_ACCESS_TOKEN'))
    handler = WebhookHandler(userdata.get('LINE_CHANNEL_SECRET'))
    return line_bot_api, handler

line_bot_api, handler = line_bot_start()

@app.route("/callback", methods=['POST'])
def callback():
    signature = request.headers['X-Line-Signature']
    body = request.get_data(as_text=True)
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        print("Invalid signature. Please check your channel access token/channel secret.")
        abort(400)
    return 'OK'

@handler.add(MessageEvent, message=TextMessage)
def handle_text(event):
    reply = TextSendMessage(text=ask_gemini(event.message.text))
    line_bot_api.reply_message(event.reply_token, reply)


# 啟動 web server
app.run()


<ipython-input-3-15a2668b0836>:50: LineBotSdkDeprecatedIn30: Call to deprecated class LineBotApi. (Use v3 class; linebot.v3.<feature>. See https://github.com/line/line-bot-sdk-python/blob/master/README.rst for more details.) -- Deprecated since version 3.0.0.
  line_bot_api = LineBotApi(userdata.get('LINE_CHANNEL_ACCESS_TOKEN'))


 * Serving Flask app '__main__'
 * Debug mode: off


<ipython-input-3-15a2668b0836>:51: LineBotSdkDeprecatedIn30: Call to deprecated class WebhookHandler. (Use 'from linebot.v3.webhook import WebhookHandler' instead. See https://github.com/line/line-bot-sdk-python/blob/master/README.rst for more details.) -- Deprecated since version 3.0.0.
  handler = WebhookHandler(userdata.get('LINE_CHANNEL_SECRET'))
INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


 * Running on http://2aa2-34-105-92-160.ngrok-free.app
 * Traffic stats available on http://127.0.0.1:4040


INFO:werkzeug:127.0.0.1 - - [18/Apr/2025 04:07:10] "GET / HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [18/Apr/2025 04:07:10] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [18/Apr/2025 04:07:23] "GET /callback HTTP/1.1" 405 -
INFO:werkzeug:127.0.0.1 - - [18/Apr/2025 04:07:37] "POST /callback HTTP/1.1" 200 -
<ipython-input-3-15a2668b0836>:77: LineBotSdkDeprecatedIn30: Call to deprecated method reply_message. (Use 'from linebot.v3.messaging import MessagingApi' and 'MessagingApi(...).reply_message(...)' instead. See https://github.com/line/line-bot-sdk-python/blob/master/README.rst for more details.) -- Deprecated since version 3.0.0.
  line_bot_api.reply_message(event.reply_token, reply)
INFO:werkzeug:127.0.0.1 - - [18/Apr/2025 04:08:36] "POST /callback HTTP/1.1" 200 -
